# Measurement of an Impulse Response with a Microphone

This notebook sketches how to measure impulse responses with a single microphone.

* Two sweeps are played, one for the analog feedback and one to the
loudspeaker.
* Two channels are recorded, the analog feedback and the microphone signal.

## Load requirements

In [ ]:
# Automatically reload the acoustics_hardware package if anything is changed
%load_ext autoreload
%autoreload 2
# fmt: off

import sys
from datetime import datetime
from pathlib import Path
from time import sleep

import numpy as np
import sounddevice as sd
from IPython.display import Audio, display, Markdown
from ipywidgets import Output
from scipy.io import savemat
from scipy.signal.waveforms import chirp

from acoustics_hardware import utils
from acoustics_hardware.processors import deconvolve

# Prepare output widgets
widget_preview_str = '<h3><center><font color="green">{}</font></center></h3>'
widget_preview_ir = Output()
widget_preview_ir.layout.border = "3px solid green"
widget_preview_rec = Output()
widget_preview_rec.layout.border = "1px solid green"

## Set up audio hardware

Adjust your settings for the individual audio hardware!

In [ ]:
# Show list of available audio devices
print(sd.query_devices())

# fmt: off
# Audio device configuration
device_name = "Orion 32"               # e.g. on macOS
# device_name = "Orion32 ASIO Driver"  # e.g. on Windows
fs = 48000                 # in Hz
device_output_ch = [1, 2]  # output channel mapping (IDs starting from 1)
device_input_ch = [1, 2]   # input channel mapping (IDs starting from 1)
# The audio device name and sampling frequency as well as input and output channels
# should be adjusted according to the utilized measurement hardware. The first channel
# in the input and output mapping thereby specify the loopback channel (the recorded
# reference signal which will be used for the deconvolution of the other channels).
# NOTE: It is currently not supported to not use a loopback channel, e.g. in case the
#       measurement hardware does not allow it. This should be implemented in the future!

# Sweep configuration
sweep_hp, sweep_lp = 50, 20e3     # in Hz
sweep_amplitude = -40             # in dB_FS
sweep_win_in_periods = 1          # in periods at the start frequency
sweep_win_out_periods = 6         # in periods at the end frequency
sweep_duration = [0.2, 2.0, 0.8]  # in s, duration of [pre-silence, sweep, post-silence]
# The frequency range and amplitude should be adjusted according to the utilized sound
# source. The number of fade-in and fade-out oscillation periods can be adjusted in
# order to not yield clicks at the start and end of the sweep (verify with the
# spectrogram of the resulting loopback measurement). The sweep duration should be
# adjusted according to the present environmental / room conditions (a longer sweep
# will yield a better signal-to-noise ratio in the deconvolved impulse responses). The
# specified post-silence time will also impose a limit on the length of the captured
# microphone signals and should therefore allow the reverberation to fully decay in all
# frequency bands.

# Deconvolution configuration
deconv_type = "lin"              # "lin" or "cyc"
deconv_hp, deconv_lp = 10, 21e3  # in Hz
deconv_inv_dyn = None            # in dB
deconv_ir_len = 2 ** 15          # in samples
deconv_win_in_len = 64           # in samples
deconv_win_out_len = 64          # in samples
deconv_ir_amp = 0                # in dB
# The deconvolution type should be chosen based on keeping ("cyc") or cutting out ("lin")
# non-harmonic distortion products from the resulting impulse responses. Either the
# frequency range or the inversion dynamic range should be restricted in order to yield
# sensible impulse responses. The length of the deconvolved impulse responses should be
# adjusted based on the room conditions to preserve the reverberation in all frequency
# bands. The resulting deconvolved impulse responses may be amplified.

# fmt: on
print(f'\nCheck audio device "{device_name}" ... ', end="")
try:
    sd.check_output_settings(
        device=device_name,
        channels=len(device_output_ch),  # as number
        samplerate=fs,                   # in Hz
    )
    sd.check_input_settings(
        device=device_name,
        channels=len(device_input_ch),   # as number
        samplerate=fs,                   # in Hz
    )
except (ValueError, sd.PortAudioError) as e:
    sys.exit(e)
print("done.")

print(f"\nGenerate sweep ... ", end="")
data_out = chirp(
    t=np.arange(np.ceil(sweep_duration[1] * fs)) / fs,  # in s
    f0=sweep_hp,                                        # in Hz
    t1=sweep_duration[1],                               # in s
    f1=sweep_lp,                                        # in Hz
    method="logarithmic",
    phi=90,  # in deg (to start with a sample at 0)
)                                                       # as [samples,]
# Apply amplitude scaling
data_out *= 10 ** (sweep_amplitude / 20)  # as factor
# Apply fade-in and fade-out window
data_out = utils.fade_signal(
    sig=data_out,
    win_in_len=int(sweep_win_in_periods * fs / sweep_hp),
    win_out_len=int(sweep_win_out_periods * fs / sweep_lp),
)
# Prepend and append zeroes
data_out = np.pad(
    array=data_out,
    pad_width=(int(sweep_duration[0] * fs), int(sweep_duration[2] * fs)),
)
data_out_is_adjusted = False
print("done.")

## Test audio hardware

Initialize the audio hardware and estimate the inherent system latency.

In [ ]:
%matplotlib inline

if data_out_is_adjusted:
    # noinspection PyUnresolvedReferences,PyUnboundLocalVariable
    display(Markdown(widget_preview_str.format(
        f"Using detected latency of {data_rec_latency} samples")))
else:
    # Determine latency if it has not been done yet
    print(f"\nRecord sweep ... ", end="")
    data_rec = sd.playrec(
        data=data_out,                    # as [samples,]
        samplerate=fs,                    # in Hz
        device=device_name,
        input_mapping=device_input_ch,    # channel numbers start from 1
        output_mapping=device_output_ch,  # channel numbers start from 1
        blocking=True,
    ).T                                   # as [channels, samples]
    print("done.")

    # Determine latency by initial zeroes in the recording
    # (this value will be used in the measurement section later)
    data_rec_latency = int(
        np.min(np.argmax(np.abs(data_rec) > 0, axis=-1))
    )  # in samples
    display(Markdown(widget_preview_str.format(
        f"Adjust for detected latency of {data_rec_latency} samples")))

    # Append zeroes to excitation signal based on determined latency
    data_out = np.pad(data_out, pad_width=(0, data_rec_latency))
    data_out_is_adjusted = True
    sleep(1)  # in s

print(f"\nRecord sweep ... ", end="")
data_rec = sd.playrec(
    data=data_out,                    # as [samples,]
    samplerate=fs,                    # in Hz
    device=device_name,
    input_mapping=device_input_ch,    # channel numbers start from 1
    output_mapping=device_output_ch,  # channel numbers start from 1
    blocking=True,
).T                                   # as [channels, samples]
print("done.")

# Cut initial zeroes based on determined latency
data_rec = data_rec[:, data_rec_latency:]  # in samples

# Print recorded peak and individual RMS levels
print(utils.generate_levels_string(data_rec))

display(Markdown("<h2><center>Excitation signal</center></h2>"))
utils.plot_data(data=data_out, fs=fs, ch_labels=device_input_ch[0])

display(Markdown("<h2><center>Recorded signals</center></h2>"))
utils.plot_data(data=data_rec, fs=fs, ch_labels=device_input_ch)
for ch in range(data_rec.shape[0]):
    display(
        Markdown(f"<h3>Channel in_{device_input_ch[ch]}</h3>"),
        Audio(data_rec[ch, :], rate=fs),
    )

data_ir = deconvolve(
    exc_sig=data_rec[:1],            # first channel as loopback
    rec_sig=data_rec[1:],
    fs=fs,                           # in Hz
    f_low=deconv_hp,                 # in Hz
    f_high=deconv_lp,                # in Hz
    res_len=deconv_ir_len / fs,      # in s
    deconv_type=deconv_type,
    inv_dyn_db=deconv_inv_dyn,       # in dB
    win_in_len=deconv_win_in_len,    # in samples
    win_out_len=deconv_win_out_len,  # in samples
)                                    # as [channels, samples]
if not data_ir.shape[0]:
    sys.exit("No deconvolved data available (only loopback channel recorded).")

display(Markdown("<h2><center>Deconvolved Impulse Responses</center></h2>"))
utils.plot_data(data=data_ir, fs=fs, ch_labels=device_input_ch[1:])
if data_ir.ndim > 1 and data_ir.shape[0] > 1:
    utils.plot_data(data=data_ir, fs=fs,
                    ch_labels=device_input_ch[1:], is_stacked=True)

## Perform measurement

In [ ]:
%matplotlib inline

# Measurement configuration
out_file_name = "data/measurement_{:s}.mat"
meas_iterations = 2  # measurement iterations per position that will be averaged

# Enter respective measurement description!
device_input_name = "Type / name of the microphone and pre-amplifier"
device_output_name = "Type / name of the loudspeaker and amplifier"
meas_room = "Name / description of the room"
meas_comment = "Further description of the environment / configuration"

# Enter manually or add later, e.g. from an `Air CO2ntrol 5000` unit!
air_temperature = None  # in degrees Celsius
air_humidity = None  # in percent, relative humidity

# Create output data path if it does not exist
Path(out_file_name).parent.mkdir(parents=True, exist_ok=True)

# Check that the recording latency has been determined
try:
    data_rec_latency
except NameError:
    sys.exit(
        'Recording latency is undefined. Execute the "Test audio hardware" section to determine it!'
    )

display(widget_preview_ir)
widget_preview_ir.clear_output()
display(widget_preview_rec)
widget_preview_rec.clear_output()

# Wait before starting the measurement
sleep(1)  # in s

# Prepare data arrays
data_rec = np.zeros(
    (
        meas_iterations,
        len(device_input_ch),
        data_out.shape[-1] - data_rec_latency,
    )
)
data_ir = np.zeros((meas_iterations, len(
    device_input_ch) - 1, deconv_ir_len))
data_time = []

for i in range(meas_iterations):
    pos_str = f"[{i + 1}/{meas_iterations}]"

    print(f"Record sweep for {pos_str} ... ", end="")
    data_time.append(datetime.now().isoformat())
    rec = sd.playrec(                     # as [iterations, channels, samples]
        data=data_out,                    # as [samples,]
        samplerate=fs,                    # in Hz
        device=device_name,
        input_mapping=device_input_ch,    # channel numbers start from 1
        output_mapping=device_output_ch,  # channel numbers start from 1
        blocking=True,
    ).T                                   # as [channels, samples]
    print("done.")

    # Cut initial zeroes based on determined latency
    data_rec[i, :] = rec[:, data_rec_latency:]  # in samples

    # Print recorded peak and individual RMS levels
    lvl_str = utils.generate_levels_string(data_rec[i])
    print(lvl_str, flush=True)

    if not np.count_nonzero(data_rec[i, :1]):
        print('Loopback signal is all zeros (deconvolution aborted).',
              file=sys.stderr, flush=True)
        continue
    if not np.count_nonzero(data_rec[i, 1:]):
        print('Microphone signal is all zeros (deconvolution aborted).',
              file=sys.stderr, flush=True)
        continue

    data_ir[i] = deconvolve(             # as [iterations, channels-1, samples]
        exc_sig=data_rec[i, :1],         # first channel as loopback
        rec_sig=data_rec[i, 1:],
        fs=fs,                           # in Hz
        f_low=deconv_hp,                 # in Hz
        f_high=deconv_lp,                # in Hz
        res_len=deconv_ir_len / fs,      # in s
        deconv_type=deconv_type,
        inv_dyn_db=deconv_inv_dyn,       # in dB
        win_in_len=deconv_win_in_len,    # in samples
        win_out_len=deconv_win_out_len,  # in samples
    )                                    # as [channels, samples]

    # Plot data preview (updated after each measurement)
    with widget_preview_rec:
        widget_preview_rec.clear_output(wait=True)
        utils.plot_data(data=data_rec[i], fs=fs,
                        ch_labels=device_input_ch)
        display(Markdown(f"<h4><center>{lvl_str}</center></h4>"))
        display(Markdown(widget_preview_str.format(
            f"Preview individual signals for {pos_str}")))
        # plot_data(data=data_ir[i], fs=fs,
        #           ch_labels=device_input_ch[1:])
        # display(Markdown(widget_preview_str.format(
        #     f"Preview individual IRs for {pos_str}")))

# Average deconvolved IRs
data_ir = np.mean(data_ir, axis=0)  # as [channels-1, samples]

# Apply desired amplification of impulse responses
data_ir *= 10 ** (deconv_ir_amp / 20)

# Plot averaged IRs preview (updated after each number of iterations)
with widget_preview_ir:
    widget_preview_ir.clear_output(wait=True)
    try:
        utils.plot_data(
            data=data_ir,
            fs=fs,
            ch_labels=device_input_ch[1:],
            is_stacked=True,
        )
        # noinspection PyUnboundLocalVariable
        display(Markdown(widget_preview_str.format(
            f"Preview averaged IRs for {pos_str}")))
    except SystemExit as e:
        display(Markdown(widget_preview_str.format(e)))

# Store data in smaller data format (single precision or integer) to lower storage space
file_name = out_file_name.format(data_time[0])
print(f'Save data to "{file_name}" ... ', end="")
savemat(
    file_name,
    {  # fmt: off
        "signal_struct": "[samples, channels, iterations]",
        "signal": data_rec.T.astype(np.float32),         # as [samples, channels, iterations]
        "ir": data_ir.T.astype(np.float32),              # as [samples, channels-1]
        "fs": np.float32(fs),                            # in Hz
        "air_temperature": air_temperature,              # in degrees Celsius
        "air_humidity": air_humidity,                    # in percent
        "ch_input": np.int16(device_input_ch),           # as list
        "ch_output": np.int16(device_output_ch),         # as list
        "dev_input": device_input_name,
        "dev_output": device_output_name,
        "dev_interface": device_name,
        "meas_room": meas_room,
        "meas_comment": meas_comment,
        "meas_time": data_time,
        "deconv_config": {  # respective `AKdeconv()` configuration
            "deconv_type": deconv_type,
            "lowPass": np.float32(deconv_lp),            # in Hz
            "highPass": np.float32(deconv_hp),           # in Hz
            "x_inv_dyn": np.float32(deconv_inv_dyn),     # in dB
            "h_trunc": np.int16(deconv_ir_len),          # in samples
            "h_fade_in": np.int16(deconv_win_in_len),    # in samples
            "h_fade_out": np.int16(deconv_win_out_len),  # in samples
        },  # fmt: on
    },
    do_compression=True,
)
print("done.")
